In [1]:
import torch
from torchvision import datasets
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
transform = v2.Compose([
    v2.Grayscale(num_output_channels=1),
    v2.Resize((128, 128)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomRotation(30),
    v2.ElasticTransform(alpha=350.0),
    v2.Normalize(mean=[0.5], std=[0.5])
])

In [3]:
dataset = datasets.ImageFolder(root="data", transform=transform)

In [4]:
dataset.classes

['galaxy', 'star']

In [5]:
totalsize = len(dataset)
train_size = int(0.7 * totalsize)
val_size = int(0.15 * totalsize)
test_size = totalsize - train_size - val_size

In [ ]:
print("f'{totalsize}{train_size}{val_size}{test_size}")

f'{totalsize}{train_size}{val_size}{test_size}


In [7]:
train_df, val_df, test_df = random_split(dataset,[train_size, val_size, test_size])

In [8]:
train_loader = DataLoader(train_df, batch_size=32, shuffle=True)
val_loader = DataLoader(val_df, batch_size=32, shuffle=False)
test_loader = DataLoader(test_df, batch_size=32, shuffle=False)

In [9]:
print(f"Total: {len(train_df + val_df + test_df)}, Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

Total: 3985, Train: 2789, Validation: 597, Test: 599


In [10]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.adaptive_pool = nn.AdaptiveMaxPool2d((8, 8))
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
model = CNN(num_classes=2)

dummy = torch.randn(1, 1, 150, 150)
out = model(dummy)

print("Output:", out.shape)

Output: torch.Size([1, 2])


In [12]:
learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion  = nn.CrossEntropyLoss()

In [13]:
num_epochs = 10

for epoch in range(10):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Geração [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Acurácia: {epoch_acc:.2f}%")

Geração [1/10] Loss: 0.6709 Acurácia: 75.98%
Geração [2/10] Loss: 0.4780 Acurácia: 76.34%
Geração [3/10] Loss: 0.4657 Acurácia: 77.55%
Geração [4/10] Loss: 0.4669 Acurácia: 76.77%
Geração [5/10] Loss: 0.4547 Acurácia: 78.81%
Geração [6/10] Loss: 0.4609 Acurácia: 77.84%
Geração [7/10] Loss: 0.4513 Acurácia: 79.42%
Geração [8/10] Loss: 0.4654 Acurácia: 77.81%
Geração [9/10] Loss: 0.4387 Acurácia: 79.53%
Geração [10/10] Loss: 0.4460 Acurácia: 79.49%


In [24]:
transform = v2.Compose([
    v2.Resize((128, 128)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(0.5),
    v2.ElasticTransform(alpha=350.0),
    v2.CenterCrop(50),
    v2.Normalize(mean=[0.5], std=[0.5])
])

In [25]:
class CNN_2(nn.Module):
    def __init__(self, num_classes):
        super(CNN_2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.adative_pool = nn.AdaptiveMaxPool2d((12, 12))
        self.fc1 = nn.Linear(144 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.adative_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [26]:
model = CNN_2(num_classes=2)

dummy = torch.randn(1, 1, 150, 150)
out = model(dummy)

print("Output:", out.shape)

Output: torch.Size([1, 2])


In [27]:
num_epochs = 10

for epoch in range(10):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    print(f"Geração [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f} Acurácia: {epoch_acc:.2f}%")

Geração [1/10] Loss: 0.7595 Acurácia: 23.59%
Geração [2/10] Loss: 0.7602 Acurácia: 23.59%
Geração [3/10] Loss: 0.7592 Acurácia: 23.59%
Geração [4/10] Loss: 0.7592 Acurácia: 23.59%
Geração [5/10] Loss: 0.7606 Acurácia: 23.59%
Geração [6/10] Loss: 0.7602 Acurácia: 23.59%
Geração [7/10] Loss: 0.7598 Acurácia: 23.59%
Geração [8/10] Loss: 0.7600 Acurácia: 23.59%
Geração [9/10] Loss: 0.7604 Acurácia: 23.59%
Geração [10/10] Loss: 0.7607 Acurácia: 23.59%
